In [12]:
import pandas as pd
import numpy as np

# Step 1: Load raw data
df_raw = pd.read_csv('world_bank_health_outcomes_raw.csv')

# Step 2: DEBUG — Print columns
print("Raw columns:")
print(df_raw.columns.tolist())

# Step 3: Extract year columns using '[YR' marker
year_cols = [col for col in df_raw.columns if '[YR' in col]
print(f"Found {len(year_cols)} year columns: {year_cols[:5]} ...")

# Step 4: Melt to long format
df_long = df_raw.melt(
    id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Value'
)

# Step 5: Extract year (e.g., from '2000 [YR2000]' → 2000)
df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int)

# Step 6: CLEAN THE 'Value' COLUMN — CRITICAL STEP
# Strip whitespace and convert to numeric, coercing errors
df_long['Value'] = (
    df_long['Value']
    .astype(str)                     # Ensure string type
    .str.strip()                     # Remove spaces
    .replace('', np.nan)             # Empty strings → NaN
    .replace('..', np.nan)           # Handle double dots (common in WB data)
    .replace('...', np.nan)
    .replace('..', np.nan)
)

# Now convert to numeric
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

# Optional: DEBUG — Check for non-finite values
print(f"Number of NaNs in Value: {df_long['Value'].isna().sum()}")

# Step 7: Pivot — use aggfunc='first' to avoid re-aggregating
df_pivot = df_long.pivot_table(
    index=['Country Name', 'Year'],
    columns='Series Name',
    values='Value',
    aggfunc='first'  # Use first non-null value (safe for clean data)
).reset_index()

# Step 8: Clean up column names
df_pivot.columns.name = None

# Step 9: Rename to clean column names
column_mapping = {
    'Life expectancy at birth, total (years)': 'Life_Expectancy',
    'Mortality rate, infant (per 1,000 live births)': 'Infant_Mortality',
    'Maternal mortality ratio (modeled estimate, per 100,000 live births)': 'Maternal_Mortality',
    'Immunization, DPT (% of children ages 12-23 months)': 'DPT_Immunization',
    'Prevalence of stunting, height for age (% of children under 5)': 'Stunting_Prev',
    'GDP per capita (current US$)': 'GDP_per_capita',
    'Population, total': 'Population'
}

# Only rename existing columns
final_columns = {}
for old, new in column_mapping.items():
    if old in df_pivot.columns:
        final_columns[old] = new
    else:
        print(f"⚠️ Not found: '{old}'")

df_pivot = df_pivot.rename(columns=final_columns)

# Step 10: Select final columns
keep_cols = ['Country Name', 'Year'] + [v for v in final_columns.values() if v in df_pivot.columns]
final_outcomes = df_pivot[keep_cols]

# Step 11: Save
final_outcomes.to_csv('world_bank_health_outcomes.csv', index=False)

# Final check
print("✅ Successfully created 'world_bank_health_outcomes.csv'")
print(f"📊 Shape: {final_outcomes.shape}")
print(f"🌍 Countries: {final_outcomes['Country Name'].nunique()}")
print(f"📅 Years: {final_outcomes['Year'].min()} to {final_outcomes['Year'].max()}")

Raw columns:
['Country Name', 'Country Code', 'Series Name', 'Series Code', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]', '2023 [YR2023]']
Found 24 year columns: ['2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]'] ...
Number of NaNs in Value: 1092
✅ Successfully created 'world_bank_health_outcomes.csv'
📊 Shape: (1248, 9)
🌍 Countries: 52
📅 Years: 2000 to 2023
